# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.113766e+02     1.533869e+00
 * time: 0.673799991607666
     1     1.112362e+01     8.055906e-01
 * time: 2.0497519969940186
     2    -1.156511e+01     9.980529e-01
 * time: 2.714574098587036
     3    -3.373089e+01     7.390935e-01
 * time: 3.7146220207214355
     4    -4.718306e+01     5.034577e-01
 * time: 4.622936964035034
     5    -5.689996e+01     1.930233e-01
 * time: 5.537729978561401
     6    -5.986305e+01     1.315256e-01
 * time: 6.195868968963623
     7    -6.095037e+01     4.993135e-02
 * time: 6.846844911575317
     8    -6.136315e+01     5.896537e-02
 * time: 7.44304895401001
     9    -6.162443e+01     3.104573e-02
 * time: 8.074876070022583
    10    -6.185118e+01     2.391195e-02
 * time: 8.741008043289185
    11    -6.201532e+01     1.567630e-02
 * time: 9.386240005493164
    12    -6.207821e+01     1.672242e-02
 * time: 10.019335985183716
    13    -6.214243e+01     1.405847e-02
 * time: 10.666745901107788
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671058
    AtomicLocal         -18.8557688
    AtomicNonlocal      14.8522669
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485365 
    Xc                  -19.3336817

    total               -62.261666455175
